In [1]:
import pandas as pd
import random
import json
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import spacy
# pip install -U spacy
# !python -m spacy download ru_core_news_md
nlp = spacy.load('ru_core_news_md')

In [3]:
df = pd.read_excel('Excel_files/Full_Poem_Dataset_9-30_0.xlsx')

In [6]:
for a in df['Author'].to_list():
    if len(a.split(' ')) != 2:
        print(a)

Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Анна Гераскина Фостер
Анна Гераскина Фостер
Анна Гераскина Фостер
Анонимно
Д.Г.
Д.Г.
Д.Г.
Екатерина Симонова / Анна Арно
Иоанна
Катя Герасимова Боски
МАКСИМ Д. ШРАЕР
Дмитрий Александрович Пригов
из книги «За Стиви Смит»
Мария Степанова / за Стиви Смит
Мария Степанова / за Стиви Смит
Перевод Алёши Прокопьева
Перевод с украинского: Полина Барскова, Остап Кинь


### Read spreadsheets

In [3]:
xls = pd.ExcelFile('Excel_files/Thesis_Poems_9-18_4.xlsx')
roarDf = pd.read_excel(xls, 'ROAR Vol. 3')
fbDf = pd.read_excel(xls, 'Facebook and Telegram')
mzDf = pd.read_excel(xls, 'Metazhurnal')
npwDf = pd.read_excel(xls, 'No War Poetry')

In [4]:
roarDf = roarDf[['Text','Author','Before or after']]
roarDf['Source'] = 'ROAR'
roarDf.head()

,Text,Author,Before or after,Source
0,Стихи о постсоветском человеке\n\n1.\nДесять з...,Александр Бараш,After,ROAR
1,2.\nПо капле выдавливать из себя \nсоветского ...,Александр Бараш,After,ROAR
2,"Дети\n\nСто шесть человек прибыли, тридцать че...",Александр Ланин,After,ROAR
3,Развод\n\nОднажды моя любимая дала пощёчину ре...,Александр Ланин,After,ROAR
4,Круциатус\n\nНевилл Лонгботтом стоит и смотрит...,Александр Ланин,After,ROAR


In [5]:
fbDf = fbDf.rename(columns={"Poem full text (copy and paste)": "Text", "Author of poem": "Author"})
fbDf = fbDf[['Text','Author','Before or after','Source']]
fbDf.head()

,Text,Author,Before or after,Source
0,"БОРИС ХЕРСОНСКИЙ\n***\nОпять же холода, окраин...",Boris Khersonsky,Before,Facebook
1,в переходе\nс Майдана Незалежности\nна Площадь...,Dmitry Strotsev,Before,Facebook
2,Начальник хора достукался \nДо сердечного прис...,Elena Fanailova,Before,Facebook
3,Начальник хора достукался \nДо сердечного прис...,Elena Fanailova,Before,Facebook
4,"Мертвая я, полумертвая я,\nИ в последнем покое...",Elena Fanailova,Before,Facebook


In [6]:
fbDf['Author'].value_counts()

Dmitry Strotsev         89
Boris Khersonsky        68
Alexander Kabanov       66
Elena Fanailova         41
Alexander Skidan        27
Cтаніслав Перфецький     1
Mikhail Eisenberg        1
Konstantin Rubakhin      1
Name: Author, dtype: int64

In [7]:
cyrillic = {
    'Boris Khersonsky' : 'Борис Херсонский',
    'Dmitry Strotsev' : 'Дмитрий Строцев',
    'Alexander Kabanov' : 'Александр Кабанов',
    'Elena Fanailova' : 'Елена Фанаилова',
    'Alexander Skidan' : 'Александр Скидан',
    'Cтаніслав Перфецький' : 'Cтаніслав Перфецький',
    'Mikhail Eisenberg' : 'Михаил Айсенберг',
    'Konstantin Rubakhin' : 'Константин Рубахин'
}
    
fbDf['Author'] = fbDf['Author'].apply(lambda x : cyrillic[x])

In [8]:
mzDf = mzDf.rename(columns={"text": "Text", "author": "Author",'sender' : 'Source'})
mzDf = mzDf[['Text','Author','Before or after','Source']]
mzDf.head()

,Text,Author,Before or after,Source
0,**2019__ #выбор_роман_осминкин \n \n между кар...,Константин Шавловский,Before,metajournal
1,сидит советский человек\n и пишет слово барабе...,Игорь Булатовский,Before,metajournal
2,отговорила проще золотая\n и начинает сложно г...,Глеб Михалев,Before,metajournal
3,В театре юного зрителя\n поставили порнографию...,Игорь Караулов,Before,metajournal
4,Из-за головы травы\n Из-за головы травы ясней\...,Андрей Гришаев,Before,metajournal


In [9]:
npwDf = npwDf[['Text','Author','Before or after']]
npwDf['Source'] = 'No War Poetry'
npwDf.head()

,Text,Author,Before or after,Source
0,СОЛДАТСКАЯ ПРОЩАЛЬНАЯ\n \n Во сне а будто бы в...,АЛЕКСАНДР АМЧИСЛАВСКИЙ,After,No War Poetry
1,Серый лёд ни капли света\n грязная вода\n отра...,АЛЕКСАНДР АМЧИСЛАВСКИЙ,After,No War Poetry
2,"Боишься, мальчик, сравнивать свой дом\n с моги...",АЛЕКСАНДР АМЧИСЛАВСКИЙ,After,No War Poetry
3,Нас тьмы и тьмы. И чёрная зима\n лежит раскину...,АЛЕКСАНДР АМЧИСЛАВСКИЙ,After,No War Poetry
4,они не знали что их завтра ждет\n война ползла...,АНАСТАСИЯ АНДРЕЕВА,After,No War Poetry


In [10]:
df = pd.concat([roarDf, fbDf, mzDf, npwDf])
print('length df',len(df))
df.head()

length df 2158


,Text,Author,Before or after,Source
0,Стихи о постсоветском человеке\n\n1.\nДесять з...,Александр Бараш,After,ROAR
1,2.\nПо капле выдавливать из себя \nсоветского ...,Александр Бараш,After,ROAR
2,"Дети\n\nСто шесть человек прибыли, тридцать че...",Александр Ланин,After,ROAR
3,Развод\n\nОднажды моя любимая дала пощёчину ре...,Александр Ланин,After,ROAR
4,Круциатус\n\nНевилл Лонгботтом стоит и смотрит...,Александр Ланин,After,ROAR


### Which analyses?

In [11]:
### Fix names

In [12]:
def standardizeName(name):
    name = name.strip()
    if len(name.split(' ')) != 2:
        return name
    first = name.split(' ')[0]
    last = name.split(' ')[1]
    first = first[0].upper() + first[1:].lower()
    last = last[0].upper() + last[1:].lower()
    full = first + ' ' + last
    return full

In [13]:
df['Author'] = df['Author'].apply(lambda x : standardizeName(x))

In [14]:
### drop NA values
df = df.dropna()

In [19]:
df['Source'].value_counts()

No War Poetry        793
metajournal          713
ROAR                 356
Facebook             228
Personal Telegram     66
Name: Source, dtype: int64

In [32]:
def removeSpace(x):
    lines = x.split('\n')
    newLines = []
    for line in lines:
        if len(line) == 0:
            newLines.append(line)
        elif line[0] == ' ' and len(line) > 1:
            newLines.append(line[1:])
        else:
            newLines.append(line)
    return '\n'.join(newLines)

In [33]:
txt = ''' Я ЗНАЮ ТО, ЧЕГО ТЫ НЕ ЗНАЕШЬ
 
 Лежат рысёнок и зайчик, у них любовь, зайчик обнимает рысёнка. Рысёнку хорошо, мур-мур-мур, и вот зайчик его спрашивает: «Отчего, рысёнок, ты так загадочно улыбаешься, как будто знаешь то, чего я не знаю?» «Я знаю то, чего ты не знаешь, зайчик, - ответил рысёнок, - но не проси меня, я тебе не скажу». «Скажи мне, рысёнок, я тоже хочу это знать», - просит зайчик. «Кое-что я знаю, зайчик, но не проси меня, я тебе не скажу». Тогда зайчик говорит рысёнку: «Если любишь меня, скажи». «Ну ладно, - ответил рысёнок, - если уж ты так этого хочешь, я скажу тебе. Я знаю то, что знают все рыси и не знает ни один зайчик. Я знаю, каковы зайчики на вкус».
 
 #выбор_Евгения_Никитина'''

print(removeSpace(txt))

Я ЗНАЮ ТО, ЧЕГО ТЫ НЕ ЗНАЕШЬ
 
Лежат рысёнок и зайчик, у них любовь, зайчик обнимает рысёнка. Рысёнку хорошо, мур-мур-мур, и вот зайчик его спрашивает: «Отчего, рысёнок, ты так загадочно улыбаешься, как будто знаешь то, чего я не знаю?» «Я знаю то, чего ты не знаешь, зайчик, - ответил рысёнок, - но не проси меня, я тебе не скажу». «Скажи мне, рысёнок, я тоже хочу это знать», - просит зайчик. «Кое-что я знаю, зайчик, но не проси меня, я тебе не скажу». Тогда зайчик говорит рысёнку: «Если любишь меня, скажи». «Ну ладно, - ответил рысёнок, - если уж ты так этого хочешь, я скажу тебе. Я знаю то, что знают все рыси и не знает ни один зайчик. Я знаю, каковы зайчики на вкус».
 
#выбор_Евгения_Никитина


In [35]:
df['Text'] = df['Text'].apply(lambda x : removeSpace(x))

In [40]:
df.to_excel('Full_Poem_Dataset_9-22_0.xlsx')

## Stats on dataset

In [49]:
df['Before or after'].value_counts()

After     1328
Before     828
Name: Before or after, dtype: int64

In [53]:
df[df['Before or after'] == 'After']['Source'].value_counts()

No War Poetry        793
ROAR                 327
Facebook             113
metajournal           84
Personal Telegram     11
Name: Source, dtype: int64

In [54]:
df[df['Before or after'] == 'Before']['Source'].value_counts()

metajournal          629
Facebook             115
Personal Telegram     55
ROAR                  29
Name: Source, dtype: int64

In [57]:
random.sample(df[df['Before or after'] == 'Before'].to_dict('records'),2)

[{'Text': '* * *\n молодым писателем-вампиром считается писатель до 400 лет. \n молодой писатель-вампир намного терпеливей, \n чем обычный писатель, который требует и вечно куда-то торопится. \n молодой писатель-вампир, после подачи рукописи \n в толстый журнал, обычно, переживает пять, \n а то и десять главредов, пока его издадут.\n может пройти двести лет, и он робко о себе напомнит. \n он не переживает, что его не издают \n и эксмо не платит гонораров, \n басинский, юзефович и быков не скажут о нём \n ни слова, плевать. \n главное для него - это признание в фейсбуке, \n бережно, он распечатывает каждый лайк, \n а по ночам навещает всех своих читателей.\n \n (Авторский блог)',
  'Author': 'Максим Матковский',
  'Before or after': 'Before',
  'Source': 'metajournal'},
 {'Text': '***\n \n ну\n вот еще\n \n но это\n еще\n не все\n \n \n https://www.sreda1.org/n13-a-m-krotkov?fbclid=IwAR1aKaGEuwzKbUhO1vuRpUuG7GdwacjfUhnqnAtMPaxgYM7ZqpReOrBMePY\n \n #выбор_ивана_полторацкого',
  'Author':

In [64]:
for period in ['Before','After']:
    with open(f'{period}_sample_9-23-22.txt','w') as f:
        poemList = random.sample(df[df['Before or after'] == period].to_dict('records'),25)
        for p in poemList:
            f.write(f"New Poem\t{p['Author']}\t{p['Source']}\n\n{p['Text']}\n\n\n\n")

## Named entities in the poems

In [11]:
%%time
records = df.to_dict('records')

NERcounter = dict()
for rec in records:
    lines = rec['Text'].split('\n')
    for txt in lines:
        doc = nlp(txt)
        for ent in doc.ents:
            lemma = ent.lemma_
            entType = ent.label_
            NERcounter.setdefault(entType, dict())
            NERcounter[entType].setdefault(lemma, 0)
            NERcounter[entType][lemma] += 1
            
lst = []
for key in NERcounter['LOC']:
    lst.append((NERcounter['LOC'][key], key))
sortedlst = sorted(lst, reverse=True)
sortedlst[:20]

CPU times: user 1min 2s, sys: 2.07 s, total: 1min 4s
Wall time: 1min 9s


## Parsing records

In [101]:
%%time
for rec in records:
    lines = rec['Text'].split('\n')
    tokens = []
    rec['Entities'] = []
    rec['Docs'] = []
    for txt in lines:
        doc = nlp(txt)
        for ent in doc.ents:
            rec['Entities'].append(ent)
        rec['Docs'].append(doc)

CPU times: user 1min 6s, sys: 3.22 s, total: 1min 9s
Wall time: 1min 19s


## Searching for long N-gram repetitions

In [98]:
%%time

ngramCounter = dict()
for n in [3,4,5]:
    for rec in records:
        lines = rec['Text'].split('\n')
        tokens = []
        for txt in lines:
            doc = nlp(txt)
            for token in doc:
                tokens.append(token)
        for i in range(len(tokens)-n+1):
            span = tuple([t.text for t in tokens[i:i+n]])
            ngramCounter.setdefault(span, {'count' : 0, 'poems' : []})
            ngramCounter[span]['count']+= 1
            ngramCounter[span]['poems'].append(rec['Text'])

CPU times: user 3min 15s, sys: 10.2 s, total: 3min 25s
Wall time: 4min 18s


In [99]:
lst = []
for span in ngramCounter:
    lst.append((ngramCounter[span]['count'],span))
sortedlst = sorted(lst, reverse=True)

### Orthographic stats

In [69]:
# word count

wordCts = dict()
lineCts = dict()
lineLengths = dict()
wordLengths = dict()

for rec in records:
    lines = rec['Text'].split('\n')
    boa = rec['Before or after']
    
    wordCts.setdefault(boa, [])
    wordLengths.setdefault(boa, [])
    lineCts.setdefault(boa, [])
    lineLengths.setdefault(boa, [])
    
    wordCt = 0
    for doc in rec['Docs']:
        for token in doc:
            wordCt += 1
            wordLengths[boa].append(len(token))
        lineLengths[boa].append(len(doc))
    lineCts[boa].append(len(rec['Docs']))
    wordCts[boa].append(wordCt)

## Word co-occurrence

In [ ]:
keywords = 'язык русский россисский украинский страна граница '

## Embeddings and tokenizer

In [9]:
import stopwordsiso as stopwords
import string
# russian, ukrainian, and english
iso_stopwords = list(stopwords.stopwords(["ru","uk","en"]))
ukstops = ['а', 'аби', 'абиде', 'абиким', 'абикого', 'абиколи', 'абикому', 'абикуди', 'абихто', 'абичий', 'абичийого', 'абичийому', 'абичим', 'абичию', 'абичия', 'абичиє', 'абичиєму', 'абичиєю', 'абичиєї', 'абичиї', 'абичиїй', 'абичиїм', 'абичиїми', 'абичиїх', 'абичого', 'абичому', 'абищо', 'абияка', 'абияке', 'абиякий', 'абияким', 'абиякими', 'абияких', 'абиякого', 'абиякому', 'абиякою', 'абиякої', 'абияку', 'абиякі', 'абиякій', 'абиякім', 'або', 'абощо', 'авжеж', 'авось', 'ага', 'ад', 'адже', 'аж', 'ажень', 'аз', 'ай', 'але', 'ало', 'амінь', 'ант', 'ану', 'ані', 'аніде', 'аніж', 'анізащо', 'аніким', 'анікого', 'анікогісінько', 'аніколи', 'анікому', 'аніскільки', 'аніхто', 'анічим', 'анічого', 'анічогісінько', 'анічому', 'аніщо', 'аніяка', 'аніяке', 'аніякий', 'аніяким', 'аніякими', 'аніяких', 'аніякого', 'аніякому', 'аніякою', 'аніякої', 'аніяку', 'аніякі', 'аніякій', 'аніякім', 'аніякісенька', 'аніякісеньке', 'аніякісенький', 'аніякісеньким', 'аніякісенькими', 'аніякісеньких', 'аніякісенького', 'аніякісенькому', 'аніякісенькою', 'аніякісенької', 'аніякісеньку', 'аніякісенькі', 'аніякісенькій', 'аніякісенькім', 'аніякісінька', 'аніякісіньке', 'аніякісінький', 'аніякісіньким', 'аніякісінькими', 'аніякісіньких', 'аніякісінького', 'аніякісінькому', 'аніякісінькою', 'аніякісінької', 'аніякісіньку', 'аніякісінькі', 'аніякісінькій', 'аніякісінькім', 'ат', 'ато', 'атож', 'ау', 'ах', 'ач', 'ачей', 'аякже', 'б', 'ба', 'багато', 'багатьма', 'багатьом', 'багатьох', 'баз', 'бай', 'бат', 'бах', 'бац', 'баш', 'бе', 'беж', 'без', 'безперервно', 'бел', 'бер', 'би', 'бир', 'бич', 'близько', 'близько від', 'бо', 'бов', 'бод', 'бодай', 'боз', 'бош', 'був', 'буває', 'буде', 'будем', 'будемо', 'будете', 'будеш', 'буду', 'будуть', 'будь', 'будь ласка', 'будьмо', 'будьте', 'була', 'були', 'було', 'бути', 'бух', 'буц', 'буцім', 'буцімто', 'бі', 'біб', 'більш', 'більше', 'біля', 'в', 'в бік', 'в залежності від', 'в міру', 'в напрямі до', 'в порівнянні з', 'в процесі', 'в результаті', 'в ролі', 'в силу', 'в сторону', 'в супроводі', 'в ході', "в ім'я", 'в інтересах', 'вад', 'важлива', 'важливе', 'важливий', 'важливі', 'вак', 'вам', 'вами', 'ван', 'вас', 'ват', 'ваш', 'ваша', 'ваше', 'вашим', 'вашими', 'ваших', 'вашого', 'вашому', 'вашою', 'вашої', 'вашу', 'ваші', 'вашій', 'вашім', 'ввесь', 'вві', 'вгору', 'вдалині', 'вед', 'верх', 'весь', 'вех', 'вже', 'вздовж', 'ви', 'виз', 'вис', 'височині', 'вище ', 'вйо', 'власне', 'властиво', 'вміти', 'внаслідок', 'вниз', 'внизу', 'во', 'вон', 'вона', 'вони', 'воно', 'восьмий', 'вперед', 'вподовж', 'впоперек', 'впритиск', 'впритул', 'впродовж', 'впрост', 'все', 'всередині', 'всею', 'вслід', 'всупереч', 'всього', 'всьому', 'всю', 'всюди', 'вся', 'всяк', 'всяка', 'всяке', 'всякий', 'всяким', 'всякими', 'всяких', 'всякого', 'всякому', 'всякою', 'всякої', 'всяку', 'всякі', 'всякій', 'всякім', 'всі', 'всій', 'всіляка', 'всіляке', 'всілякий', 'всіляким', 'всілякими', 'всіляких', 'всілякого', 'всілякому', 'всілякою', 'всілякої', 'всіляку', 'всілякі', 'всілякій', 'всілякім', 'всім', 'всіма', 'всіх', 'всією', 'всієї', 'втім', 'ві', 'віг', 'від', 'від імені', 'віддалік від', 'відколи', 'відносно', 'відповідно', 'відповідно до', 'відсотків', 'відтепер', 'відтоді', 'він', 'вісім', 'вісімнадцятий', 'вісімнадцять', 'віт', 'віф', 'віх', 'віц', 'віщо', 'віщось', 'г', 'га', 'гав', 'гаразд', 'ге', 'гез', 'гем', 'геп', 'гет', 'геть', 'гех', 'ги', 'гик', 'гир', 'гич', 'гм', 'го', 'говорив', 'гог', 'гоп', 'гоц', 'гу', 'гуп', 'д', 'да', 'давай', 'давати', 'давно', 'далеко', 'далеко від', 'далі', 'даром', 'два', 'двадцятий', 'двадцять', 'дванадцятий', 'дванадцять', 'двох', 'дві', 'де', "дев'ятий", "дев'ятнадцятий", "дев'ятнадцять", "дев'ять", 'дедалі', 'деким', 'декого', 'деколи', 'декому', 'декотра', 'декотре', 'декотрий', 'декотрим', 'декотрими', 'декотрих', 'декотрого', 'декотрому', 'декотрою', 'декотрої', 'декотру', 'декотрі', 'декотрій', 'декотрім', 'декілька', 'декільком', 'декількома', 'декількох', 'декім', 'десь', 'десятий', 'десять', 'дехто', 'дечий', 'дечийого', 'дечийому', 'дечим', 'дечию', 'дечия', 'дечиє', 'дечиєму', 'дечиєю', 'дечиєї', 'дечиї', 'дечиїй', 'дечиїм', 'дечиїми', 'дечиїх', 'дечого', 'дечому', 'дечім', 'дещо', 'деяка', 'деяке', 'деякий', 'деяким', 'деякими', 'деяких', 'деякого', 'деякому', 'деякою', 'деякої', 'деяку', 'деякі', 'деякій', 'деякім', 'деінде', 'для', 'до', 'добре', 'довго', 'довкола', 'довкіл', 'дог', 'доки', 'допоки', 'допіру', 'досить', 'досі', 'дотепер', 'доти', 'другий', 'друго', 'дуже', 'дякую', 'дійсно', 'діл', 'е', 'еге', 'еж', 'ей', 'ерг', 'ест', 'ет', 'ех', 'еч', 'ж', 'же', 'жоден', 'жодна', 'жодне', 'жодний', 'жодним', 'жодними', 'жодних', 'жодного', 'жодному', 'жодною', 'жодної', 'жодну', 'жодні', 'жодній', 'жоднім', 'жоднісінька', 'жоднісіньке', 'жоднісінький', 'жоднісіньким', 'жоднісінькими', 'жоднісіньких', 'жоднісінького', 'жоднісінькому', 'жоднісінькою', 'жоднісінької', 'жоднісіньку', 'жоднісінькі', 'жоднісінькій', 'жоднісінькім', 'жуз', 'з', 'з метою', 'з нагоди', 'з приводу', 'з розрахунку на', 'з-за', 'з-над', 'з-перед', 'з-поза', 'з-поміж', 'з-понад', 'з-поперед', 'з-посеред', 'з-проміж', 'з-під', 'з-серед', 'за', 'за винятком', 'за допомогою', 'за посередництвом', 'за рахунок', 'завгодно', 'завдяки', 'завжди', 'завше', 'задля', 'зазвичай', 'зайнята', 'зайнятий', 'зайнято', 'зайняті', 'залежно', 'залежно від', 'замість', 'занадто', 'заради', 'зараз', 'зас', 'зате', 'збоку', 'збоку від', 'зважаючи на', 'зверх ', 'зверху', 'звичайно', 'звиш', 'звідки', 'звідкилясь', 'звідкись', 'звідкіль', 'звідкіля', 'звідкілясь', 'звідси', 'звідсіль', 'звідсіля', 'звідти', 'звідтіль', 'звідтіля', 'звідусюди', 'звідусіль', 'звідціля', 'згідно з', 'здається', 'здовж', 'зем', 'зет', 'ззаду', 'зиз', 'зик', 'значить', 'знову', 'зо', 'зовсім', 'зсередини', 'зух', 'зі', 'зіс', 'и', 'ич', 'й', 'ймовірно', 'йно', 'йо', 'його', 'йой', 'йол', 'йому', 'йор', 'йот', 'йох', 'к', 'каже', 'каз', 'кар', 'каф', 'ках', 'ке', 'кед', 'кет', 'кеш', 'кив', 'кий', 'кил', 'ким', 'кимось', 'кимсь', 'ких', 'киш', 'коб', 'коби', 'кого', 'когось', 'кожен', 'кожна', 'кожне', 'кожний', 'кожним', 'кожними', 'кожних', 'кожного', 'кожному', 'кожною', 'кожної', 'кожну', 'кожні', 'кожній', 'кожнім', 'кожнісінька', 'кожнісіньке', 'кожнісінький', 'кожнісіньким', 'кожнісінькими', 'кожнісіньких', 'кожнісінького', 'кожнісінькому', 'кожнісінькою', 'кожнісінької', 'кожнісіньку', 'кожнісінькі', 'кожнісінькій', 'кожнісінькім', 'коли', 'колись', 'коло', 'кому', 'комусь', 'котра', 'котрась', 'котре', 'котресь', 'котрий', 'котрийсь', 'котрим', 'котрими', 'котримись', 'котримось', 'котримсь', 'котрих', 'котрихось', 'котрихсь', 'котрого', 'котрогось', 'котрому', 'котромусь', 'котрою', 'котроюсь', 'котрої', 'котроїсь', 'котру', 'котрусь', 'котрі', 'котрій', 'котрійсь', 'котрім', 'котрімсь', 'котрісь', 'коц', 'коч', 'коштом', 'край', 'краще', 'кру', 'круг', 'кругом', 'крю', 'кря', 'крізь', 'крім', 'куди', 'кудись', 'кудою', 'кілька', 'кільком', 'кількома', 'кількох', 'кім', 'кімось', 'кімсь', 'кінець', 'л', 'лаж', 'лап', 'лас', 'лат', 'ле', 'ледве', 'ледь', 'лет', 'лиш', 'лише', 'лишень', 'лум', 'луп', 'лут', 'льє', 'люди', 'людина', 'ля', 'лі', 'ліворуч від', 'лік', 'лім', 'м', 'мабуть', 'майже', 'мало', 'мати', 'мац', 'ме', 'меж', 'мене', 'менше', 'мені', 'мерсі', 'мет', 'мжа', 'ми', 'мимо ', 'миру', 'мит', 'мною', 'мо', 'мов', 'мовби', 'мовбито', 'могла', 'могли', 'могло', 'мого', 'могти', 'мож', 'може', 'можем', 'можемо', 'можете', 'можеш', 'можна', 'можу', 'можуть', 'можіть', 'мой', 'мол', 'мою', 'моя', 'моє', 'моєму', 'моєю', 'моєї', 'мої', 'моїй', 'моїм', 'моїми', 'моїх', 'му', 'мі', 'міг', 'між', 'мій', 'мільйонів', 'н', 'на', 'на адресу', 'на базі', 'на благо', 'на випадок', 'на відміну від', 'на засадах', 'на знак', 'на зразок', 'на користь', 'на кшталт', 'на межі', 'на основі', 'на противагу', 'на підставі', 'на честь', 'на чолі', 'на ґрунті', 'навколо', 'навкруг', 'навкруги ', 'навкіл', 'навпаки', 'навперейми', 'навпроти', 'навіть', 'навіщо', 'навіщось', 'нагорі', 'над', 'надо', 'надовкола', 'надокола', 'наді', 'назавжди', 'назад', 'назустріч', 'най', 'найбільш', 'нам', 'нами', 'наоколо ', 'наокруг ', 'наокруги ', 'наокіл', 'наперед', 'напередодні', 'напереді', 'наперекір', 'напереріз', 'наприкінці', 'напроти', 'нарешті', 'нарівні з', 'нас', 'насеред', 'насподі', 'наспід', 'настрічу', 'насупроти', 'насупротив ', 'нате', 'наче', 'начеб', 'начебто', 'наш', 'наша', 'наше', 'нашим', 'нашими', 'наших', 'нашого', 'нашому', 'нашою', 'нашої', 'нашу', 'наші', 'нашій', 'нашім', 'не', 'не до', 'не можна', 'неабичим', 'неабичого', 'неабичому', 'неабищо', 'небагато', 'небагатьма', 'небагатьом', 'небагатьох', 'небудь', 'невважаючи', 'невже', 'недалеко', 'недалеко від', 'неж', 'незалежно від', 'незважаючи', 'незважаючи на', 'ней', 'немає', 'немов', 'немовби', 'немовбито', 'неначе', 'неначебто', 'неподалеку', 'неподалеку від', 'неподалечку', 'неподалечку від', 'неподалік', 'неподалік від', 'нерідко', 'нех', 'нехай', 'нещодавно', 'нею', 'неї', 'нижче', 'низько', 'ник', 'ним', 'ними', 'них', 'нич', 'но', 'ну', 'нуг', 'нуд', 'нум', 'нумо', 'нумте', 'ньо', 'нього', 'ньому', 'ню', 'нюх', 'ня', 'няв', 'ні', 'ніби', 'ніби-то', 'нібито', 'ніде', 'ніж', 'нізащо', 'нізвідки', 'нізвідкіля', 'ній', 'ніким', 'нікого', 'нікогісінько', 'ніколи', 'нікому', 'нікотра', 'нікотре', 'нікотрий', 'нікотрим', 'нікотрими', 'нікотрих', 'нікотрого', 'нікотрому', 'нікотрою', 'нікотрої', 'нікотру', 'нікотрі', 'нікотрій', 'нікотрім', 'нікуди', 'нім', 'нінащо', 'ніскільки', 'ніт', 'ніхто', 'нічий', 'нічийна', 'нічийне', 'нічийний', 'нічийним', 'нічийними', 'нічийних', 'нічийного', 'нічийному', 'нічийною', 'нічийної', 'нічийну', 'нічийні', 'нічийній', 'нічийнім', 'нічийого', 'нічийому', 'нічим', 'нічию', 'нічия', 'нічиє', 'нічиєму', 'нічиєю', 'нічиєї', 'нічиї', 'нічиїй', 'нічиїм', 'нічиїми', 'нічиїх', 'нічого', 'нічому', 'ніщо', 'ніяк', 'ніяка', 'ніяке', 'ніякий', 'ніяким', 'ніякими', 'ніяких', 'ніякого', 'ніякому', 'ніякою', 'ніякої', 'ніяку', 'ніякі', 'ніякій', 'ніякім', 'ніякісінька', 'ніякісіньке', 'ніякісінький', 'ніякісіньким', 'ніякісінькими', 'ніякісіньких', 'ніякісінького', 'ніякісінькому', 'ніякісінькою', 'ніякісінької', 'ніякісіньку', 'ніякісінькі', 'ніякісінькій', 'ніякісінькім', 'о', 'об', 'обабіч', 'обаполи', 'обидва', 'обр', 'обік', 'обіруч', 'обіч', 'ов', 'од', 'один', 'одинадцятий', 'одинадцять', 'одна', 'однак', 'одначе', 'одне', 'одним', 'одними', 'одних', 'одно', 'одного', 'одного разу', 'одному', 'одною', 'одної', 'одну', 'одні', 'одній', 'однім', 'однією', 'однієї', 'ож', 'ой', 'окрай', 'окроме', 'округ', 'округи', 'окрім', 'окіл', 'ом', 'он', 'онде', 'онно', 'оно', 'оподаль', 'оподаль від', 'оподалік', 'оподалік від', 'опостін', 'опостінь', 'опроче', 'опріч', 'опріче', 'опісля', 'осе', 'оскільки', 'особливо', 'осторонь', 'ось', 'осісьо', 'от', 'ота', 'отак', 'отака', 'отаке', 'отакий', 'отаким', 'отакими', 'отаких', 'отакого', 'отакому', 'отакою', 'отакої', 'отаку', 'отакі', 'отакій', 'отакім', 'отакісінька', 'отакісіньке', 'отакісінький', 'отакісіньким', 'отакісінькими', 'отакісіньких', 'отакісінького', 'отакісінькому', 'отакісінькою', 'отакісінької', 'отакісіньку', 'отакісінькі', 'отакісінькій', 'отакісінькім', 'отам', 'оте', 'отже', 'отим', 'отими', 'отих', 'ото', 'отого', 'отож', 'отой', 'отому', 'отою', 'отої', 'отсе', 'оттак', 'отто', 'оту', 'отут', 'оті', 'отій', 'отім', 'отією', 'отієї', 'ох', 'оце', 'оцей', 'оцим', 'оцими', 'оцих', 'оцього', 'оцьому', 'оцю', 'оця', 'оці', 'оцій', 'оцім', 'оцією', 'оцієї', 'п', "п'я", "п'ятий", "п'ятнадцятий", "п'ятнадцять", "п'ять", 'па', 'пад', 'пак', 'пек', 'перед', 'передо', 'переді', 'перетака', 'перетаке', 'перетакий', 'перетаким', 'перетакими', 'перетаких', 'перетакого', 'перетакому', 'перетакою', 'перетакої', 'перетаку', 'перетакі', 'перетакій', 'перетакім', 'перший', 'пиж', 'плі', 'по', 'поблизу', 'побік', 'побіля', 'побіч', 'поверх', 'повз', 'повздовж', 'повинно', 'повище', 'повсюди', 'повсюдно', 'подаль від', 'подалі від', 'подекуди', 'подеяка', 'подеяке', 'подеякий', 'подеяким', 'подеякими', 'подеяких', 'подеякого', 'подеякому', 'подеякою', 'подеякої', 'подеяку', 'подеякі', 'подеякій', 'подеякім', 'подовж', 'подібно до', 'поз', 'поза', 'позад', 'позаду', 'позата', 'позате', 'позатим', 'позатими', 'позатих', 'позатого', 'позатой', 'позатому', 'позатою', 'позатої', 'позату', 'позаті', 'позатій', 'позатім', 'позатією', 'позатієї', 'позаяк', 'поздовж', 'поки', 'покрай', 'покіль', 'помежи', 'помимо', 'поміж', 'помість', 'понад', 'понадо', 'понаді', 'понижче', 'пообіч', 'поодаль від', 'поодалік від', 'поперед', 'попереду', 'поперек', 'попліч', 'попри', 'попросту', 'попід', 'пора', 'поруч', 'поряд', 'поряд з', 'порівняно з', 'посеред', 'посередині', 'потрібно', 'потім', 'поуз', 'початку', 'почерез', 'праворуч від', 'пред', 'предо', 'преді', 'прекрасно', 'прецінь', 'при', 'притому', 'причому', 'причім', 'про', 'проз', 'промеж', 'проміж', 'просто', 'проте', 'проти', 'против', 'противно', 'протягом', 'пря', 'пріч', 'пхе', 'пху', 'пі', 'пів', 'півперек', 'під', 'під знаком', 'під приводом', 'під час', 'підо', 'пізніше', 'пім', 'пір', 'після', 'р', 'ради', 'раз', 'разом з', 'разу', 'рано', 'раніш', 'раніш від', 'раніше', 'раніше від', 'раптом', 'ре', 'рет', 'риж', 'рим', 'рип', 'роб', 'року', 'років', 'рос', 'рох', 'році', 'рус', 'рух', 'руч', 'рік', 'с', 'саж', 'саз', 'сак', 'сам', 'сама', 'саме', 'сами', 'самий', 'самим', 'самими', 'самих', 'само', 'самого', 'самому', 'самою', 'самої', 'саму', 'самі', 'самій', 'самім', 'сап', 'сас', 'свого', 'свою', 'своя', 'своє', 'своєму', 'своєю', 'своєї', 'свої', 'своїй', 'своїм', 'своїми', 'своїх', 'свій', 'се', 'себе', 'себто', 'сей', 'сен', 'серед', 'середи', 'середу', 'сеч', 'си', 'сив', 'сиг', 'сиз', 'сик', 'сиріч', 'сих', 'сказав', 'сказала', 'сказати', 'скрізь', 'скільки', 'скільки-то', 'скількись', 'скільком', 'скількома', 'скількомась', 'скількомось', 'скількомсь', 'скількох', 'скількохось', 'скількохсь', 'сли', 'слідом за', 'соб', 'собою', 'собі', 'соп', 'спасибі', 'спереду', 'спочатку', 'справ', 'справді', 'став', 'стосовно', 'стільки', 'стільком', 'стількома', 'стількох', 'су', 'судячи з', 'супроти', 'супротив', 'суть', 'суч', 'суш', 'сьогодні', 'сьомий', 'сюди', 'ся', 'сяг', 'сяк', 'сяка', 'сяке', 'сякий', 'сяким', 'сякими', 'сяких', 'сякого', 'сякому', 'сякою', 'сякої', 'сяку', 'сякі', 'сякій', 'сякім', 'сям', 'сі', 'сім', 'сімнадцятий', 'сімнадцять', 'сіп', 'т', 'та', 'таж', 'так', 'така', 'таке', 'такенна', 'такенне', 'такенний', 'такенним', 'такенними', 'такенних', 'такенного', 'такенному', 'такенною', 'такенної', 'такенну', 'такенні', 'такенній', 'такеннім', 'таки', 'такий', 'таким', 'такими', 'таких', 'такого', 'також', 'такому', 'такою', 'такої', 'таку', 'такі', 'такій', 'такім', 'такісінька', 'такісіньке', 'такісінький', 'такісіньким', 'такісінькими', 'такісіньких', 'такісінького', 'такісінькому', 'такісінькою', 'такісінької', 'такісіньку', 'такісінькі', 'такісінькій', 'такісінькім', 'тал', 'там', 'тамки', 'тамта', 'тамте', 'тамтим', 'тамтими', 'тамтих', 'тамтого', 'тамтой', 'тамтому', 'тамтою', 'тамтої', 'тамту', 'тамті', 'тамтій', 'тамтім', 'тамтією', 'тамтієї', 'тар', 'тат', 'таш', 'тва', 'твого', 'твою', 'твоя', 'твоє', 'твоєму', 'твоєю', 'твоєї', 'твої', 'твоїй', 'твоїм', 'твоїми', 'твоїх', 'твій', 'те', 'тебе', 'тег', 'теж', 'тем', 'тепер', 'теперечки', 'тес', 'теф', 'теє', 'ти', 'тик', 'тил', 'тим', 'тими', 'тисяч', 'тих', 'то', 'тобою', 'тобто', 'тобі', 'того', 'тоді', 'тож', 'той', 'тол', 'тому', 'тому що', 'тот', 'тощо', 'тою', 'тої', 'тра', 'тре', 'треба', 'третій', 'три', 'тринадцятий', 'тринадцять', 'трохи', 'тс', 'тсс', 'ту', 'туди', 'тудою', 'туп', 'тут', 'тутеньки', 'тутечки', 'тутки', 'туф', 'туц', 'тю', 'тюг', 'тюп', 'тяг', 'тяж', 'тям', 'тяп', 'ті', 'тій', 'тільки', 'тім', 'тією', 'у', 'у бік', 'у вигляді', 'у випадку', 'у відповідності до', 'у відповідь на', 'у залежності від', "у зв'язку з", 'у міру', 'у напрямі до', 'у порівнянні з', 'у процесі', 'у результаті', 'у ролі', 'у силу', 'у сторону', 'у супроводі', 'у ході', 'ув', 'увесь', 'уві', 'угу', 'уже', 'узбіч', 'уздовж', 'укр', 'ум', 'унаслідок', 'униз', 'унизу', 'унт', 'уперед', 'уподовж', 'упоперек', 'упритиск до', 'упритул до', 'упродовж', 'упрост', 'ус', 'усе', 'усередині', 'услід', 'услід за', 'усупереч', 'усього', 'усьому', 'усю', 'усюди', 'уся', 'усяк', 'усяка', 'усяке', 'усякий', 'усяким', 'усякими', 'усяких', 'усякого', 'усякому', 'усякою', 'усякої', 'усяку', 'усякі', 'усякій', 'усякім', 'усі', 'усій', 'усіляка', 'усіляке', 'усілякий', 'усіляким', 'усілякими', 'усіляких', 'усілякого', 'усілякому', 'усілякою', 'усілякої', 'усіляку', 'усілякі', 'усілякій', 'усілякім', 'усім', 'усіма', 'усіх', 'усією', 'усієї', 'утім', 'ух', 'ф', "ф'ю", 'фа', 'фаг', 'фай', 'фат', 'фе', 'фед', 'фез', 'фес', 'фет', 'фзн', 'фоб', 'фот', 'фра', 'фру', 'фу', 'фук', 'фур', 'фус', 'фіш', 'х', 'ха', 'хаз', 'хай', 'хап', 'хат', 'хащ', 'хе', 'хет', 'хи', 'хиб', 'хм', 'хо', 'хов', 'хол', 'хон', 'хоп', 'хор', 'хотіти', 'хоч', 'хоча', 'хочеш', 'хро', 'хрю', 'хто', 'хтось', 'ху', 'хуз', 'хук', 'хух', 'хху', 'хіба', 'ц', 'це', 'цебто', 'цей', 'цеп', 'ци', 'цим', 'цими', 'цир', 'цих', 'цло', 'цоб', 'цок', 'цоп', 'цор', 'цс', 'цсс', 'цуг', 'цур', 'цуц', 'цього', 'цьому', 'цю', 'цюк', 'ця', 'цяв', 'цяп', 'ці', 'цід', 'цій', 'цім', 'ціною', 'цією', 'цієї', 'ч', 'чал', 'чар', 'час', 'часто', 'частіше', 'часу', 'чах', 'чей', 'чень', 'через', 'четвертий', 'чи', 'чий', 'чийого', 'чийогось', 'чийому', 'чийомусь', 'чийсь', 'чик', 'чим', 'чимось', 'чимсь', 'чир', 'численна', 'численне', 'численний', 'численним', 'численними', 'численних', 'численні', 'чию', 'чиюсь', 'чия', 'чиясь', 'чиє', 'чиєму', 'чиємусь', 'чиєсь', 'чиєю', 'чиєюсь', 'чиєї', 'чиєїсь', 'чиї', 'чиїй', 'чиїйсь', 'чиїм', 'чиїми', 'чиїмись', 'чиїмось', 'чиїмсь', 'чиїсь', 'чиїх', 'чиїхось', 'чиїхсь', 'чля', 'чого', 'чогось', 'чом', 'чому', 'чомусь', 'чон', 'чоп', 'чортзна', 'чос', 'чотири', 'чотирнадцятий', 'чотирнадцять', 'чу', 'чум', 'чур', 'чш', 'чім', 'чімось', 'чімсь', 'чіт', 'ш', 'ша', 'шаг', 'шал', 'шам', 'шво', 'шед', 'шен', 'шиз', 'шир', 'шляхом', 'шостий', 'шістнадцятий', 'шістнадцять', 'шість', 'щ', 'ще', 'щем', 'щеп', 'щип', 'щир', 'що', 'щоб', 'щоби', 'щодо', 'щойно', 'щоправда', 'щось', 'щі', 'ь', 'ю', 'юз', 'юн', 'юнь', 'юс', 'ют', 'юхт', 'я', 'яв', 'яд', 'яз', 'язь', 'як', 'яка', 'якась', 'якби', 'яке', 'якесь', 'який', 'якийсь', 'яким', 'якими', 'якимись', 'якимось', 'якимсь', 'яких', 'якихось', 'якихсь', 'якого', 'якогось', 'якому', 'якомусь', 'якось', 'якою', 'якоюсь', 'якої', 'якоїсь', 'якраз', 'яку', 'якусь', 'якщо', 'які', 'якій', 'якійсь', 'якім', 'якімсь', 'якісь', 'ял', 'ям', 'ян', 'янь', 'яо', 'яп', 'ярл', 'ясь', 'ять', 'є', 'єр', 'єси', 'і', 'ібн', 'ід', 'із', 'із-за', 'із-під', 'іззаду', 'ізм', 'ізсередини', 'ік', 'ікс', 'ікт', "ім'я", 'імовірно', 'інакша', 'інакше', 'інакший', 'інакшим', 'інакшими', 'інакших', 'інакшого', 'інакшому', 'інакшою', 'інакшої', 'інакшу', 'інакші', 'інакшій', 'інакшім', 'інколи', 'іноді', 'інша', 'інше', 'інший', 'іншим', 'іншими', 'інших', 'іншого', 'іншому', 'іншою', 'іншої', 'іншу', 'інші', 'іншій', 'іншім', 'інь', 'іч', 'іще', 'ї', 'їдь', 'їй', 'їм', 'їх', 'їхнього', 'їхньому', 'їхньою', 'їхньої', 'їхню', 'їхня', 'їхнє', 'їхні', 'їхній', 'їхнім', 'їхніми', 'їхніх', 'її', 'ґ']
all_stops = iso_stopwords + ukstops
stops_and_punct = list(string.punctuation) + all_stops

In [10]:
def poemTokenizer(poem):
    lines = poem.split('\n')
    tokens = []
    for line in lines:
        doc = nlp(line)
        for token in doc:
            if token.has_vector and token.text not in stops_and_punct:
                tokens.append(token.lemma_)
    return tokens

### Convert lines to document vectors

In [11]:
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
# vectorizer_model = CountVectorizer(ngram_range=(1, 1),lowercase=False,tokenizer=tokenizer)
# X = vectorizer_model.fit_transform(sample)

### Document sample

In [18]:
authorsOfInterest = ['Elena Fanailova','Boris Khersonsky','Dmitry Strotsev']
recsOfInterest = [r for r in records if r['Author'] in authorsOfInterest]
sample = [r['Text'] for r in recsOfInterest]
labels = [authorsOfInterest.index(r['Author']) for r in recsOfInterest]

In [20]:
len(sample)

198

In [15]:
%%time

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(sample, show_progress_bar=True)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

CPU times: user 53.7 s, sys: 5.93 s, total: 59.6 s
Wall time: 1min 19s


In [16]:
embeddings.shape

(198, 384)

### Crashes every time on TSNE and UMAP

In [22]:
from sklearn.manifold import TSNE
from sklearn.datasets import load_iris
from numpy import reshape
import seaborn as sns
import pandas as pd  

In [ ]:
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(embeddings) 

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 198 samples in 0.003s...
[t-SNE] Computed neighbors for 198 samples in 0.056s...
[t-SNE] Computed conditional probabilities for sample 198 / 198
[t-SNE] Mean sigma: 0.212411
